In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/aprix-
[nltk_data]     vitor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
file = 'public_data/train/track_a/ptbr.csv'
train = pd.read_csv(file)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226 entries, 0 to 2225
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2226 non-null   object
 1   text      2226 non-null   object
 2   Anger     2226 non-null   int64 
 3   Disgust   2226 non-null   int64 
 4   Fear      2226 non-null   int64 
 5   Joy       2226 non-null   int64 
 6   Sadness   2226 non-null   int64 
 7   Surprise  2226 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 139.2+ KB


In [4]:
train.head()

,id,text,Anger,Disgust,Fear,Joy,Sadness,Surprise
0,ptbr_train_track_a_00001,"minha vó me disse que era frango e eu comi, ti...",0,0,0,0,1,0
1,ptbr_train_track_a_00002,Está e a nossa deputada Benedita linda guerrei...,0,0,0,1,0,0
2,ptbr_train_track_a_00003,só falta as roupas kkkkkkkkkkk,0,0,0,1,0,0
3,ptbr_train_track_a_00004,Eu tmb. Comecei a sair de casa agora (fui pela...,0,0,0,0,1,0
4,ptbr_train_track_a_00005,Peço a Deus que nossos dirigentes tenham realm...,0,0,0,0,0,0


### Pré-processamento dos dados
1. Case folding
2. Remover stop words
3. Remover acentuação e pontuação


In [5]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords[:20]

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele']

In [6]:
train["clean_text"] = train["text"].copy()
# Case folding
train["clean_text"] = train["clean_text"].str.lower()
# Remover stopwords
train["clean_text"] = train["clean_text"].replace({r"\b" + stopword + r"\b": "" for stopword in stopwords}, regex=True)
# Remover acentuação e pontuação
train["clean_text"] = train["clean_text"].str.replace(r"[\.!?\\\-.,]", "", regex=True)
train["clean_text"] = train["clean_text"].str.replace(r"\s+", " ", regex=True)


In [7]:
train["text"], train["clean_text"]

(0       minha vó me disse que era frango e eu comi, ti...
 1       Está e a nossa deputada Benedita linda guerrei...
 2                          só falta as roupas kkkkkkkkkkk
 3       Eu tmb. Comecei a sair de casa agora (fui pela...
 4       Peço a Deus que nossos dirigentes tenham realm...
                               ...                        
 2221              Eu acho que o CAP vai surpreender hein.
 2222    23:59 - Lula sabia de toda a corrupção no seu ...
 2223    O Brasil precisa URGENTE de pessoas sérias e c...
 2224    Sera que só eu acho que ta passando da hora de...
 2225                                     falta só 2 porra
 Name: text, Length: 2226, dtype: object,
 0         vó disse frango comi gosto frango mto inocente 
 1        deputada benedita linda guerreira parabéns ju...
 2                                falta roupas kkkkkkkkkkk
 3        tmb comecei sair casa agora ( primeira vez ci...
 4       peço deus dirigentes realmente iluminação toma...
              

### Adicionar emoção neutra

Para casos que não tem nenhuma emoção na linha, adicionamos uma nova emoção: neutro

In [8]:
BASE_EMOTIONS = ["Anger", "Disgust", "Fear", "Joy", "Sadness", "Surprise"]

train["Neutral"] = 0
no_emotions_mask = train[BASE_EMOTIONS].sum(axis=1) == 0
train.loc[no_emotions_mask, "Neutral"] = 1

### Finetuning do modelo

In [9]:
from transformers import AutoModel

/home/aprix-vitor/.cache/pypoetry/virtualenvs/pln-trabalho-final-A9EDKhyv-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model = "neuralmind/bert-base-portuguese-cased"

In [ ]:
AutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29794, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False